## Quiz

## Review

1. Find some data!
2. Load and look at your data
  * thing1
  * thing2
3. Consider cleaning, transforming and/or normalizing your data
  * thing1
  * thing2
  * thing3
  * thing4
4. Look at your data some more and consider feature selection, feature engineering, dimensionality reduction
  * *correlations*
  * *covariance matrix*
  * *PCA*
5. Model
  * thing1 (in three variations!)

## Data

Today we will keep working with the set of Craigslist listings for used cars.

Same code as Wednesday!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import scipy.linalg as sp_la

In [ ]:
# these will be our columns
columns = ["price", "year", "manufacturer", "model", "condition", "fuel", "odometer", "title_status", "transmission"]
# this will contain our converters
colValues = {}

# first we load our data as strings so we can define the converters
data = np.array(np.genfromtxt('data/vehicles.csv', delimiter=',', usecols=(1,2,3,4,5,7,8,9,11), skip_header=1, dtype=str, encoding='utf-8'))  

# make a list of the unique values in each column of our data
for colIndex in range(data.shape[1]):
    colValues[colIndex] = np.unique(data[:, colIndex]).tolist()
    print(colIndex, colValues[colIndex])

# fix up some of these ones we know are ordered
colValues[columns.index('condition')] = ['new', 'like new', 'excellent', 'good', 'fair', 'salvage']
colValues[columns.index('title_status')] = ['clean', 'lien', 'rebuilt', 'salvage', 'parts only', 'missing']

# map values to their indices in the list of unique values
def converter(x, colIndex):
    return colValues[colIndex].index(x)

data = np.array(np.genfromtxt('data/vehicles.csv', delimiter=',', usecols=(1,2,3,4,5,7,8,9,11), converters={3: lambda x: converter(x, 2), 4: lambda x: converter(x, 3), 5: lambda x: converter(x, 4), 7: lambda x: converter(x,5), 9: lambda x: converter(x, 7), 11: lambda x: converter(x, 8)}, skip_header=1, dtype=int, encoding='utf-8'))  

Let's calculate *correlations* between price and the other variables. (Remind me what correlation values vary between?)

In [ ]:
for i in range(len(columns)):
    print(columns[i], np.corrcoef(data[:, 0], data[:, i], rowvar=True)[0,1])

# Covariance

As we can see from the correlation analysis above, features (variables) in a data set may be related to each other.

The *covariance matrix* of a data set tells us about the first order relationships between different features. If $A$ is the matrix corresponding to our data set of $N$ data points for each of which we have $M$ features, and $\bar{A_i}$ is the mean of the $i$th feature, then covariance matrix $C$ is:
    $$C_{i,j} = \sum_{k=1}^N \frac{(A_{k,i} - \bar{A_i})(A_{k,j} - \bar{A_j})}{N-1}$$
    
The covariance matrix has the variance of each feature along its diagonal, and the remaining entries are the *covariances* of pairs of features, ie how much they vary together. If they vary together, then they are related to each other - some information is shared between them.

Questions:
* If the covariance is close to 0, then what is true of the pair of features?
* If the covariance is big, then what is true of the pair of features?
* Correlations can be positive or negative; what about covariances?

We can calculate covariance using matrix multiplication:
* First, center the data: $A_c = A - \bar{A}$
* Then, calculate $C$: $C = \frac{1}{N-1} A_c^TA_c$

Let's look at some toy examples (h/t Stephanie Taylor!)

## Example 1

These two variables co-vary.

In [ ]:
A = np.array([[1,2,3,0], [1.1, 2.1, 3, 0.5]]).T
A

In [ ]:
# First, center the data
def centerData(data):
    return data - data.mean(axis=0)

# Then, calculate C
def covarMatrix(data):
    centered = centerData(data)
    return (centered.T@centered) / (len(centered)-1) # or centered.shape[0] - 1

print(centerData(A))
print(covarMatrix(A))

*What do we observe about the variance of X? of Y? What about the covariance?*

## Example 2

This is just like before by the values of $Y$ are now negative.

In [ ]:
A = np.array([[1,2,3,0], [-1.1, -2.1, -3, -0.5]]).T
A

In [ ]:

# First, center the data
print(centerData(A))

# Then, calculate C
print(covarMatrix(A))

*What do we observe about the variance of X? of Y? What about the covariance?*

## Example 3

The values of $X$ and $Y$ are now random.

In [ ]:
A = np.array([np.random.standard_normal(4), np.random.standard_normal(4)]).T
A

In [ ]:
# First, center the data
print(centerData(A))

# Then, calculate C
print(covarMatrix(A))

*What do we observe about the variance of X? of Y? What about the covariance?*

## On our data

In [ ]:
C = covarMatrix(data)

# I want to see it better!
print(pd.DataFrame(C))

*What do we observe?*

## Warning

This is a thing that will calculate the covariance matrix for you, but you *may not use it* for your project: np.cov

# The curse of dimensionality

When data is very multidimensional, everything becomes slower and more complicated:
* How can you look at your data? There's too much there!
* How can you normalize your data? Without being able to look at it, what normalizations would you know to apply?
* How can you fit a regression to your data? Which variables would you choose?

In fact, data in a high dimensional space just is different:
* there are more extreme values. In a 2 dimensional unit square, the probability that a random point is within 0.001 of the border is 0.004, but in a 10000 dimensional unit hypercube, the probability is > 0.999.
* the distances between points are bigger. In a 2 dimensional unit square, the average distance between two points is about 0.5. In a 3 dimensional unit cube, it's 0.66. But in a 10000 dimensional unit hypercube, it's about 408.25.

Here's the thing though: generally, only a small fraction of variables in a data set suffice to model the whole. For example, very often many features do covary. So we can use covariance information to *project* our data to a much smaller dimensional space and analyze the data in that space instead. The technique we will use for that is *principal component analysis* (PCA).